<div align="center">
<a href="https://rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/RapidFire - Blue bug -white text.svg" width="115"></a>
<a href="https://discord.gg/6vSTtncKNN"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/discord-button.svg" width="145"></a>
<a href="https://oss-docs.rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/documentation-button.svg" width="125"></a>
<br/>
Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/RapidFireAI/rapidfireai">GitHub</a></i> ⭐
<br/>
To install RapidFire AI on your own machine, see the <a href="https://oss-docs.rapidfire.ai/en/latest/walkthrough.html">Install and Get Started</a> guide in our docs.
</div>

### Enable all metric loggers

In [ ]:
import os
os.environ["RF_MLFLOW_ENABLED"] = "true"
os.environ["RF_TENSORBOARD_ENABLED"] = "true"
os.environ["RF_TRACKIO_ENABLED"] = "true"


### RapidFire AI Tutorial Use Case: SFT for Customer Support Q&A Chatbot

In [ ]:
from rapidfireai import Experiment
from rapidfireai.automl import (
    List,
    RFGridSearch,
    RFModelConfig,
    RFLoraConfig,
    RFSFTConfig,
)

### Load Dataset and Specify Train and Eval Partitions

In [ ]:
from datasets import load_dataset

dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Select a subset of the dataset for demo purposes
train_dataset = dataset["train"].select(range(4))
eval_dataset = dataset["train"].select(range(10, 11))
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

### Define Data Processing Function

In [ ]:
def sample_formatting_function(example):
    """Format the dataset for GPT-2 while preserving original fields"""
    return {
        "text": f"Question: {example['instruction']}\nAnswer: {example['response']}",
        "instruction": example["instruction"],  # Keep original
        "response": example["response"],  # Keep original
    }


# Apply formatting to datasets
eval_dataset = eval_dataset.map(sample_formatting_function)
train_dataset = train_dataset.map(sample_formatting_function)

### Initialize Experiment

In [ ]:
# Every experiment instance must be uniquely named
experiment = Experiment(experiment_name="exp1-chatqa-tiny", mode="fit")

### Define Custom Eval Metrics Function

In [ ]:
def sample_compute_metrics(eval_preds):
    """Optional function to compute eval metrics based on predictions and labels"""
    predictions, labels = eval_preds

    # Standard text-based eval metrics: Rouge and BLEU
    import evaluate

    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")

    rouge_output = rouge.compute(
        predictions=predictions, references=labels, use_stemmer=True
    )
    rouge_l = rouge_output["rougeL"]
    bleu_output = bleu.compute(predictions=predictions, references=labels)
    bleu_score = bleu_output["bleu"]

    return {
        "rougeL": round(rouge_l, 4),
        "bleu": round(bleu_score, 4),
    }

### Define Multi-Config Knobs for Model, LoRA, and SFT Trainer using RapidFire AI Wrapper APIs

In [ ]:
# GPT-2 specific LoRA configs - different module names!
peft_configs_tiny = List(
    [
        RFLoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["c_attn"],  # GPT-2 combines Q,K,V in c_attn
            bias="none",
        ),
        RFLoraConfig(
            r=8,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["c_attn", "c_proj"],  # c_attn (QKV) + c_proj (output)
            bias="none",
        ),
    ]
)

# 2 configs with GPT-2
config_set_tiny = List(
    [
        RFModelConfig(
            model_name="gpt2",  # Only 124M params
            peft_config=peft_configs_tiny,
            training_args=RFSFTConfig(
                learning_rate=5e-4,  # Low lr for more stability
                lr_scheduler_type="linear",
                per_device_train_batch_size=1,  # Effective bs = 4
                max_steps=8,  # Raise this to see more learning
                logging_steps=2,
                eval_strategy="steps",
                eval_steps=4,
                per_device_eval_batch_size=1,
                fp16=True,
                report_to="none",  # Disables wandb
            ),
            model_type="causal_lm",
            model_kwargs={
                "device_map": "auto",
                "torch_dtype": "float16",  # Explicit fp16
                "use_cache": False,
            },
            formatting_func=sample_formatting_function,
            compute_metrics=sample_compute_metrics,
            generation_config={
                "max_new_tokens": 128,  # Reduced from 256
                "temperature": 0.7,
                "top_p": 0.9,
                "top_k": 40,
                "repetition_penalty": 1.1,
                "pad_token_id": 50256,  # GPT-2's EOS token
            },
        )
    ]
)

#### Define Model Creation Function for All Model Types Across Configs

In [ ]:
def sample_create_model(model_config):
    """Function to create model object with GPT-2 adjustments"""
    from transformers import AutoModelForCausalLM, AutoTokenizer

    model_name = model_config["model_name"]
    model_type = model_config["model_type"]
    model_kwargs = model_config["model_kwargs"]

    if model_type == "causal_lm":
        model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
    else:
        # Default to causal LM
        model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # GPT-2 specific: Set pad token (GPT-2 doesn't have one by default)
    if "gpt2" in model_name.lower():
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"  # GPT-2 works better with left padding
        model.config.pad_token_id = model.config.eos_token_id

    return (model, tokenizer)

#### Generate Config Group

In [ ]:
# Simple grid search across all sets of config knob values = 4 combinations in total
config_group = RFGridSearch(configs=config_set_tiny, trainer_type="SFT")

### Show Tensorboard

In [ ]:
%load_ext tensorboard

from rapidfireai.utils.metric_rfmetric_manager import RFMetricLogger
metric_loggers = RFMetricLogger.get_default_metric_loggers(experiment_name=experiment.experiment_name)
tensorboard_log_dir = metric_loggers.get("rf_tensorboard", {}).get("config", {}).get("log_dir", ".")

print(f"TensorBoard logs will be saved to: {tensorboard_log_dir}")
%tensorboard --logdir {tensorboard_log_dir}

### Run Multi-Config Training

In [ ]:
# Launch training of all configs in the config_group with swap granularity of 4 chunks
experiment.run_fit(
    config_group,
    sample_create_model,
    train_dataset,
    eval_dataset,
    num_chunks=2,
    seed=42,
)

### End Current Experiment

In [ ]:
experiment.end()

<div align="center">
<a href="https://rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/RapidFire - Blue bug -white text.svg" width="115"></a>
<a href="https://discord.gg/6vSTtncKNN"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/discord-button.svg" width="145"></a>
<a href="https://oss-docs.rapidfire.ai/"><img src="https://raw.githubusercontent.com/RapidFireAI/rapidfireai/main/docs/images/documentation-button.svg" width="125"></a>
<br/>
Thanks for trying RapidFire AI! ⭐ <i>Star us on <a href="https://github.com/RapidFireAI/rapidfireai">GitHub</a></i> ⭐
</div>